## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-23-13-05-13 +0000,nyt,4 Members of Illinois Politician Darren Bailey...,https://www.nytimes.com/2025/10/23/us/politics...
1,2025-10-23-13-00-27 +0000,nypost,Portland Trail Blazers coach Chauncey Billups ...,https://nypost.com/2025/10/23/us-news/portland...
2,2025-10-23-13-00-05 +0000,nyt,"Under Trump, National Symphony Opens Concerts ...",https://www.nytimes.com/2025/10/23/arts/music/...
3,2025-10-23-13-00-00 +0000,missionlocal,Baseball player,https://missionlocal.org/2025/10/sf-photos-bas...
4,2025-10-23-13-00-00 +0000,missionlocal,Live updates: Trump deploys federal agents to ...,https://missionlocal.org/2025/10/ice-agents-sf...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
23,trump,49
95,new,25
42,up,15
147,sanctions,14
206,russian,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
261,2025-10-22-20-58-00 +0000,wsj,President Trump has privately told allies he b...,https://www.wsj.com/politics/elections/trump-z...,140
164,2025-10-23-03-10-21 +0000,nypost,Entire White House East Wing will be demolishe...,https://nypost.com/2025/10/22/us-news/entire-w...,132
37,2025-10-23-11-45-10 +0000,bbc,Trump says Putin talks 'don't go anywhere' as ...,https://www.bbc.com/news/articles/cd6758pn6ylo...,122
69,2025-10-23-10-29-48 +0000,wapo,E.U. joins U.S. with new Russia sanctions as T...,https://www.washingtonpost.com/world/2025/10/2...,116
208,2025-10-22-23-43-10 +0000,nyt,Trump Imposes Sanctions on Russian Oil Compani...,https://www.nytimes.com/2025/10/22/us/politics...,105


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
261,140,2025-10-22-20-58-00 +0000,wsj,President Trump has privately told allies he b...,https://www.wsj.com/politics/elections/trump-z...
222,68,2025-10-22-22-47-41 +0000,wapo,U.S. imposes sanctions on Russian oil companie...,https://www.washingtonpost.com/world/2025/10/2...
162,59,2025-10-23-03-25-16 +0000,nypost,"Mamdani breaks a sweat, fails to give specific...",https://nypost.com/2025/10/22/us-news/mamdani-...
164,58,2025-10-23-03-10-21 +0000,nypost,Entire White House East Wing will be demolishe...,https://nypost.com/2025/10/22/us-news/entire-w...
169,49,2025-10-23-02-41-32 +0000,nypost,Mom of 4 shot dead by ‘obsessed’ ex-boyfriend ...,https://nypost.com/2025/10/22/world-news/young...
348,46,2025-10-22-16-10-00 +0000,wsj,A U.S. Plan Splits Gaza in Two—One Zone Contro...,https://www.wsj.com/world/middle-east/a-u-s-pl...
281,45,2025-10-22-20-16-45 +0000,nypost,GM embraces AI with plans to use ‘eyes-off’ dr...,https://nypost.com/2025/10/22/business/gm-embr...
305,44,2025-10-22-18-47-38 +0000,nypost,"US military bombs 8th, 9th suspected drug-traf...",https://nypost.com/2025/10/22/us-news/us-milit...
344,42,2025-10-22-16-24-00 +0000,wsj,"Fed officials, who are unable to receive U.S. ...",https://www.wsj.com/economy/central-banking/fe...
356,41,2025-10-22-15-43-53 +0000,nypost,Why NJ gov poll that shows Mikie Sherrill up 5...,https://nypost.com/2025/10/22/us-news/why-nj-g...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
